In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from fastDamerauLevenshtein import damerauLevenshtein

### read data

In [2]:
data = open('data.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
data

'pocgkr\ncgwpcfks\nkfsfkspwg\npfkswkfsococg\nwptgkfsoc\npwfkg\npowcer\ncoprerk\nwopofkkskf\nopwpwgcoc\npocrfksfks\ncgpogpoc\npwocgfkfsrere\nocwgrk\nwpgsfk\npwofsg\npocpocwgkfs\npocgwg\npwgockfsfksret\nocpwgfkskfsrere\ncopwgwpresk\npwterfkf\npocwggpfkfksrer\nwpofcgkrpwgfksocr\ncwpfksor\npwfksorc\npocgwkfkfser\npowctgcog\npowgcpo\npwfkt\npwokfkrg\ncopcogpgwfksr\npgsfkskfg\nwgptocopwg\npowcgre\nperkscwgtf\npcgworkfkfs\npopocgkr\npwtgcfkfksr\npocwkf\npgkfkf\ncwopg\npowcgkfkfr\nwpgfksk\npocwcpoere\npwkf\ncptgo\npocfkswgr\npowcfkfk\npwgfksfk\ngpwfks\ncgwpw\ncwopgtrekfksksf\npocsfkw\nocpwgfkf\npwgterkfsoc\npocgwrkfs\ntgcwpro\npocgkfskfr\npgcow\nopwg\nwpgcowpcfksre\nre\nptwgfks\npocrfkfk\npockr\npockfskfswgre\ntgpcowpgpwcog\ngwpfrkst\npfkfksg\npwogsfkg\npwkf\ngwpocoskfsfk\nwtpcockfkf\nwpgfkfkre\npococwtrfkfkg\npwocfksfksr\npwgerks\npoctfkfkrwg\npwfksocr\ntnpzc\ntnpcz\ntnpzc\ntnzpc\npmz\nxmbzd\npImz\nmzxp\npbz\npgz\npnzga\ndnzgdp\npndz\nnzpdgds\nmcddxp\npcfkdddaag\ndagpk\npc\ngkfadp\npfgds\nbcs

### generate list of sequences

In [44]:
data_split = data.splitlines()

### remove newline chars

In [5]:
data2 = data.replace('\n','')
data2

'pocgkrcgwpcfkskfsfkspwgpfkswkfsococgwptgkfsocpwfkgpowcercoprerkwopofkkskfopwpwgcocpocrfksfkscgpogpocpwocgfkfsrereocwgrkwpgsfkpwofsgpocpocwgkfspocgwgpwgockfsfksretocpwgfkskfsrerecopwgwpreskpwterfkfpocwggpfkfksrerwpofcgkrpwgfksocrcwpfksorpwfksorcpocgwkfkfserpowctgcogpowgcpopwfktpwokfkrgcopcogpgwfksrpgsfkskfgwgptocopwgpowcgreperkscwgtfpcgworkfkfspopocgkrpwtgcfkfksrpocwkfpgkfkfcwopgpowcgkfkfrwpgfkskpocwcpoerepwkfcptgopocfkswgrpowcfkfkpwgfksfkgpwfkscgwpwcwopgtrekfksksfpocsfkwocpwgfkfpwgterkfsocpocgwrkfstgcwpropocgkfskfrpgcowopwgwpgcowpcfksrereptwgfkspocrfkfkpockrpockfskfswgretgpcowpgpwcoggwpfrkstpfkfksgpwogsfkgpwkfgwpocoskfsfkwtpcockfkfwpgfkfkrepococwtrfkfkgpwocfksfksrpwgerkspoctfkfkrwgpwfksocrtnpzctnpcztnpzctnzpcpmzxmbzdpImzmzxppbzpgzpnzgadnzgdppndznzpdgdsmcddxppcfkdddaagdagpkpcgkfadppfgdsbcsfgkappbsfkcgapbgafkcsgpbcksfapbfkscgapbsgkafcpbcskfgapsfkagcbpfkscagbpfskagcbpskfcagbpfkscagbpsfkgacbpfkbscgapcgafkbsgpabkfscpabkgcfsgcpabkfspasbkfgcpabcgksfsfkpcgabspgbkfcaspakgfcbcgbkpasffkspcbagpkf

In [6]:
vocabulary = sorted(set(data2))
#vocabulary.remove('\n')
len(vocabulary)

20

### Vectorize text

In [7]:
example_texts = ['abcdefg', 'xhz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [8]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocabulary))

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[3, 4, 5, 6, 7, 8, 9], [20, 10, 21]]>

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xhz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training samples + targets

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(data2, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1412,), dtype=int64, numpy=array([15, 14,  5, ...,  6,  9,  4])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [19]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

p
o
c
g
k
r
c
g
w
p


In [20]:
seq_length = 8
examples_per_epoch = len(data)//(seq_length+1)

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor([b'p' b'o' b'c' b'g' b'k' b'r' b'c' b'g' b'w'], shape=(9,), dtype=string)


In [22]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'pocgkrcgw'
b'pcfkskfsf'
b'kspwgpfks'
b'wkfsococg'
b'wptgkfsoc'


In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'pocgkrcg'
Target: b'ocgkrcgw'


### Create training batches

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 8), (64, 8)), types: (tf.int64, tf.int64)>

### Build model

In [27]:
# Length of the vocabulary in chars
vocab_size = len(vocabulary)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [29]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Try out model

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 8, 22) # (batch_size, sequence_length, vocab_size)


In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  5632      
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  22550     
Total params: 3,966,486
Trainable params: 3,966,486
Non-trainable params: 0
_________________________________________________________________


In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'bfgsckcp'

Next Char Predictions:
 b'hetogkhg'


### Train model

In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 8, 22)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.0910795


In [36]:
tf.exp(mean_loss).numpy()

22.000814

In [37]:
model.compile(optimizer='adam', loss=loss)

In [91]:
epochs = 1000

In [92]:
history = model.fit(dataset, epochs=epochs)

Epoch 1/1000
2/2 [==============================] - 0s 70ms/step - loss: 0.4015
Epoch 2/1000
2/2 [==============================] - 0s 74ms/step - loss: 0.3923
Epoch 3/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3905
Epoch 4/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3907
Epoch 5/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3832
Epoch 6/1000
2/2 [==============================] - 0s 72ms/step - loss: 0.3736
Epoch 7/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3819
Epoch 8/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3731
Epoch 9/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3778
Epoch 10/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3654
Epoch 11/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3759
Epoch 12/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3702
Epoch 13/1000
2/2 [==================

2/2 [==============================] - 0s 67ms/step - loss: 0.3178
Epoch 103/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3224
Epoch 104/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3185
Epoch 105/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3289
Epoch 106/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3260
Epoch 107/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3265
Epoch 108/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3248
Epoch 109/1000
2/2 [==============================] - 0s 71ms/step - loss: 0.3226
Epoch 110/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3226
Epoch 111/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3213
Epoch 112/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3239
Epoch 113/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3195
Epoch 114/1000
2/2 [===========

2/2 [==============================] - 0s 63ms/step - loss: 0.3219
Epoch 203/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3180
Epoch 204/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3200
Epoch 205/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3122
Epoch 206/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3114
Epoch 207/1000
2/2 [==============================] - 0s 59ms/step - loss: 0.3172
Epoch 208/1000
2/2 [==============================] - 0s 59ms/step - loss: 0.3151
Epoch 209/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3150
Epoch 210/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3153
Epoch 211/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3253
Epoch 212/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3172
Epoch 213/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3237
Epoch 214/1000
2/2 [===========

2/2 [==============================] - 0s 70ms/step - loss: 0.3166
Epoch 303/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3197
Epoch 304/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3174
Epoch 305/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3199
Epoch 306/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3152
Epoch 307/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3156
Epoch 308/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3221
Epoch 309/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3146
Epoch 310/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3134
Epoch 311/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3231
Epoch 312/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3184
Epoch 313/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3161
Epoch 314/1000
2/2 [===========

2/2 [==============================] - 0s 63ms/step - loss: 0.3164
Epoch 403/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3182
Epoch 404/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3128
Epoch 405/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3184
Epoch 406/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3230
Epoch 407/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3152
Epoch 408/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3206
Epoch 409/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3207
Epoch 410/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3200
Epoch 411/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3125
Epoch 412/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3146
Epoch 413/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3165
Epoch 414/1000
2/2 [===========

2/2 [==============================] - 0s 61ms/step - loss: 0.3129
Epoch 503/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3150
Epoch 504/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3121
Epoch 505/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3181
Epoch 506/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3126
Epoch 507/1000
2/2 [==============================] - 0s 72ms/step - loss: 0.3143
Epoch 508/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3133
Epoch 509/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3168
Epoch 510/1000
2/2 [==============================] - 0s 67ms/step - loss: 0.3169
Epoch 511/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3131
Epoch 512/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3153
Epoch 513/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3125
Epoch 514/1000
2/2 [===========

2/2 [==============================] - 0s 61ms/step - loss: 0.3101
Epoch 603/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3129
Epoch 604/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3219
Epoch 605/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3130
Epoch 606/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3144
Epoch 607/1000
2/2 [==============================] - 0s 72ms/step - loss: 0.3161
Epoch 608/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3160
Epoch 609/1000
2/2 [==============================] - 0s 70ms/step - loss: 0.3160
Epoch 610/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3117
Epoch 611/1000
2/2 [==============================] - 0s 75ms/step - loss: 0.3114
Epoch 612/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3115
Epoch 613/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3124
Epoch 614/1000
2/2 [===========

2/2 [==============================] - 0s 66ms/step - loss: 0.3168
Epoch 703/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3173
Epoch 704/1000
2/2 [==============================] - 0s 59ms/step - loss: 0.3091
Epoch 705/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3159
Epoch 706/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3216
Epoch 707/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3064
Epoch 708/1000
2/2 [==============================] - 0s 74ms/step - loss: 0.3116
Epoch 709/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3115
Epoch 710/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3135
Epoch 711/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3140
Epoch 712/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3128
Epoch 713/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3114
Epoch 714/1000
2/2 [===========

2/2 [==============================] - 0s 70ms/step - loss: 0.3132
Epoch 803/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3110
Epoch 804/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3132
Epoch 805/1000
2/2 [==============================] - 0s 73ms/step - loss: 0.3159
Epoch 806/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3144
Epoch 807/1000
2/2 [==============================] - 0s 63ms/step - loss: 0.3144
Epoch 808/1000
2/2 [==============================] - 0s 69ms/step - loss: 0.3072
Epoch 809/1000
2/2 [==============================] - 0s 70ms/step - loss: 0.3187
Epoch 810/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.3128
Epoch 811/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3158
Epoch 812/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3138
Epoch 813/1000
2/2 [==============================] - 0s 69ms/step - loss: 0.3181
Epoch 814/1000
2/2 [===========

2/2 [==============================] - 0s 75ms/step - loss: 0.3122
Epoch 903/1000
2/2 [==============================] - 0s 60ms/step - loss: 0.3140
Epoch 904/1000
2/2 [==============================] - 0s 64ms/step - loss: 0.3168
Epoch 905/1000
2/2 [==============================] - 0s 62ms/step - loss: 0.3153
Epoch 906/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3122
Epoch 907/1000
2/2 [==============================] - 0s 68ms/step - loss: 0.3105
Epoch 908/1000
2/2 [==============================] - 0s 70ms/step - loss: 0.3158
Epoch 909/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3143
Epoch 910/1000
2/2 [==============================] - 0s 73ms/step - loss: 0.3122
Epoch 911/1000
2/2 [==============================] - 0s 65ms/step - loss: 0.3189
Epoch 912/1000
2/2 [==============================] - 0s 59ms/step - loss: 0.3137
Epoch 913/1000
2/2 [==============================] - 0s 66ms/step - loss: 0.3168
Epoch 914/1000
2/2 [===========

### Generate prediction

In [93]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]) 
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [94]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [95]:
states = None
next_char = tf.constant(['p'])
result = [next_char]

for n in range(8):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

print(result[0].numpy().decode('utf-8'))

pffksrere


### Prequential method

In [96]:
# test generation of predictions in loop for all first chars in list

results = [[] for line in data_split]

for line in range(0, len(data_split)):
    states = None
    next_char = tf.constant([data_split[line][0]])
    results[line] = [next_char]
    
    for n in range(0, len(data_split[line]) - 1):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        results[line].append(next_char)
    results[line] = tf.strings.join(results[line])
    results[line] = results[line][0].numpy().decode('utf-8')

In [97]:
# check if predicted length = observed length
[len(line) for line in results] == [len(line) for line in data_split]

True

In [98]:
# generate prediction for next char, compare to observed, determine error - until length of observed is reached

results = [[] for line in data_split]
errors = [[] for x in data_split]

for line in range(0, len(data_split)):
    i = 1
    states = None
    
    while i < len(data_split[line]):
        next_char = tf.constant([data_split[line][:i]])
        results[line] = [next_char]
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        results[line].append(next_char)
        predicted = tf.strings.join(results[line])
        predicted = predicted[0].numpy().decode('utf-8')
        observed = data_split[line][:i+1]
        error = 1 - damerauLevenshtein(predicted, observed)
        #print(predicted, observed, error)
        
        errors[line].append(error)
        i +=1
        
    results[line] = tf.strings.join(results[line])
    results[line] = results[line][0].numpy().decode('utf-8')

In [99]:
[np.mean(error) for error in errors]

[0.09,
 0.12636054421768708,
 0.14112103174603174,
 0.0981129981129981,
 0.1921626984126984,
 0.175,
 0.2566666666666667,
 0.13055555555555556,
 0.10555555555555556,
 0.12549603174603174,
 0.11155202821869488,
 0.24540816326530615,
 0.1314814814814815,
 0.12333333333333332,
 0.11666666666666667,
 0.29000000000000004,
 0.09754329004329004,
 0.18333333333333332,
 0.12096877481492865,
 0.11351862423290993,
 0.12603535353535356,
 0.18333333333333335,
 0.11610235003092147,
 0.05682852686529157,
 0.21683673469387754,
 0.16207482993197278,
 0.03654729109274564,
 0.2077876984126984,
 0.23214285714285712,
 0.175,
 0.17261904761904764,
 0.14460470085470084,
 0.20362103174603177,
 0.16340388007054674,
 0.20833333333333334,
 0.1567901234567901,
 0.18948773448773445,
 0.1547619047619048,
 0.13353535353535353,
 0.19,
 0.18333333333333332,
 0.20833333333333334,
 0.1521604938271605,
 0.23214285714285712,
 0.12173721340388008,
 0.2777777777777778,
 0.2583333333333333,
 0.1619543650793651,
 0.0654761904

In [100]:
np.mean([np.mean(error) for error in errors])

0.17865622366414707

In [101]:
np.median([np.mean(error) for error in errors])

0.16666666666666669